In [2]:
from py2neo import Graph, Node, Relationship, NodeMatcher

class CustomerProductGraph:
    def __init__(self, uri="neo4j+s://d5800584.databases.neo4j.io", user="neo4j", password="wPQh0QgH3kzqb8iBSAenCL54ZX7suzKy4vt4_jdj6tU"):
        self.graph = Graph(uri, auth=(user, password))
        self.matcher = NodeMatcher(self.graph)

    # Clear graph database
    def clear_graph(self):
        self.graph.run("MATCH (n) DETACH DELETE n")
        print("Graph cleared.")

    # Add customers in batch
    def add_customers(self, customer_names):
        tx = self.graph.begin()
        for name in customer_names:
            customer = Node("Customer", name=name)
            tx.merge(customer, "Customer", "name")
            print(f"Customer '{name}' added.")
        tx.commit()

    # Add products in batch
    def add_products(self, product_names):
        tx = self.graph.begin()
        for name in product_names:
            product = Node("Product", name=name)
            tx.merge(product, "Product", "name")
            print(f"Product '{name}' added.")
        tx.commit()

    # Add categories in batch
    def add_categories(self, category_names):
        tx = self.graph.begin()
        for name in category_names:
            category = Node("Category", name=name)
            tx.merge(category, "Category", "name")
            print(f"Category '{name}' added.")
        tx.commit()

    # Add purchase relationships
    def add_purchase(self, customer_name, product_name):
        customer = self.matcher.match("Customer", name=customer_name).first()
        product = self.matcher.match("Product", name=product_name).first()
        if customer and product:
            purchase = Relationship(customer, "PURCHASED", product)
            self.graph.merge(purchase)
            print(f"Customer '{customer_name}' purchased product '{product_name}'.")
        else:
            print(f"Customer or product not found!")

    # Add belongs_to relationships between product and category
    def add_product_category(self, product_name, category_name):
        product = self.matcher.match("Product", name=product_name).first()
        category = self.matcher.match("Category", name=category_name).first()
        if product and category:
            rel = Relationship(product, "BELONGS_TO", category)
            self.graph.merge(rel)
            print(f"Product '{product_name}' belongs to category '{category_name}'.")
        else:
            print(f"Product or Category not found!")

    # Find products purchased by a customer
    def find_products_purchased_by_customer(self, customer_name):
        query = """
        MATCH (c:Customer)-[:PURCHASED]->(p:Product)
        WHERE c.name = $customer_name
        RETURN p.name AS product
        """
        results = self.graph.run(query, customer_name=customer_name).data()
        return [record['product'] for record in results]

    # Recommend products to a customer based on category
    def recommend_products_by_category(self, customer_name):
        query = """
        MATCH (c:Customer)-[:PURCHASED]->(p:Product)-[:BELONGS_TO]->(cat:Category)<-[:BELONGS_TO]-(rec:Product)
        WHERE c.name = $customer_name AND NOT (c)-[:PURCHASED]->(rec)
        RETURN DISTINCT rec.name AS recommendation
        LIMIT 5
        """
        results = self.graph.run(query, customer_name=customer_name).data()
        return [record['recommendation'] for record in results]

# Example usage
if __name__ == "__main__":
    graph = CustomerProductGraph()

    # Clear graph (optional)
    graph.clear_graph()

    # Adding Customers, Products, and Categories
    customers = ["Alice", "Bob", "Charlie"]
    products = ["Laptop", "Phone", "Tablet", "Smartwatch"]
    categories = ["Electronics", "Gadgets"]

    graph.add_customers(customers)
    graph.add_products(products)
    graph.add_categories(categories)

    # Create relationships
    graph.add_purchase("Alice", "Laptop")
    graph.add_purchase("Bob", "Phone")
    graph.add_purchase("Charlie", "Tablet")

    graph.add_product_category("Laptop", "Electronics")
    graph.add_product_category("Phone", "Electronics")
    graph.add_product_category("Tablet", "Electronics")
    graph.add_product_category("Smartwatch", "Gadgets")

    # Query products purchased by Alice
    products_by_alice = graph.find_products_purchased_by_customer("Alice")
    print("Products purchased by Alice:", products_by_alice)

    # Recommend products for Alice based on category
    recommendations = graph.recommend_products_by_category("Alice")
    print("Product recommendations for Alice:", recommendations)


Graph cleared.
Customer 'Alice' added.
Customer 'Bob' added.
Customer 'Charlie' added.
Product 'Laptop' added.


C:\Users\Chandan D\AppData\Local\Temp\ipykernel_24832\3250550836.py:20: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


Product 'Phone' added.
Product 'Tablet' added.
Product 'Smartwatch' added.


C:\Users\Chandan D\AppData\Local\Temp\ipykernel_24832\3250550836.py:29: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


Category 'Electronics' added.
Category 'Gadgets' added.


C:\Users\Chandan D\AppData\Local\Temp\ipykernel_24832\3250550836.py:38: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


Customer 'Alice' purchased product 'Laptop'.
Customer 'Bob' purchased product 'Phone'.
Customer 'Charlie' purchased product 'Tablet'.
Product 'Laptop' belongs to category 'Electronics'.
Product 'Phone' belongs to category 'Electronics'.
Product 'Tablet' belongs to category 'Electronics'.
Product 'Smartwatch' belongs to category 'Gadgets'.
Products purchased by Alice: ['Laptop']
Product recommendations for Alice: ['Phone', 'Tablet']
